#Importing Libraries

In [1]:
#basic import
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
import warnings


#Data Preprocessing

In [2]:
#importing dataset
dataset = pd.read_csv('student-mat.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)
print(y)

[]
['G3' '6' '6' '10' '15' '10' '15' '11' '6' '19' '15' '9' '12' '14' '11'
 '16' '14' '14' '10' '5' '10' '15' '15' '16' '12' '8' '8' '11' '15' '11'
 '11' '12' '17' '16' '12' '15' '6' '18' '15' '11' '13' '11' '12' '18' '11'
 '9' '6' '11' '20' '14' '7' '13' '13' '10' '11' '13' '10' '15' '15' '9'
 '16' '11' '11' '9' '9' '10' '15' '12' '6' '8' '16' '15' '10' '5' '14'
 '11' '10' '10' '11' '10' '5' '12' '11' '6' '15' '10' '8' '6' '14' '10'
 '7' '8' '18' '6' '10' '14' '10' '15' '10' '14' '8' '5' '17' '14' '6' '18'
 '11' '8' '18' '13' '16' '19' '10' '13' '19' '9' '16' '14' '13' '8' '13'
 '15' '15' '13' '13' '8' '12' '11' '9' '0' '18' '0' '0' '12' '11' '0' '0'
 '0' '0' '12' '15' '0' '9' '11' '13' '0' '11' '0' '11' '0' '10' '0' '14'
 '10' '0' '12' '8' '13' '10' '15' '12' '0' '7' '0' '10' '7' '12' '10' '16'
 '0' '14' '0' '16' '10' '0' '9' '9' '11' '6' '9' '11' '8' '12' '17' '8'
 '12' '11' '11' '15' '9' '10' '13' '9' '8' '10' '14' '15' '16' '10' '18'
 '10' '16' '10' '10' '6' '11' '9' '7' '13' '10'

In [4]:
dataset.head()

student-mat
school sex age address famsize Pstatus Medu Fedu Mjob    Fjob     reason guardian traveltime studytime failures schoolsup famsup paid activities nursery higher internet romantic famrel freetime goout Dalc Walc health absences G1 G2          G3
GP     F   18  U       GT3     A       4    4    at_home teacher  course mother   2          2         0        yes       no     no   no         yes     yes    no       no       4      3        4     1    1    3      6        5  6            6
           17  U       GT3     T       1    1    at_home other    course father   1          2         0        no        yes    no   no         no      yes    yes      no       5      3        3     1    1    3      4        5  5            6
           15  U       LE3     T       1    1    at_home other    other  mother   1          2         3        yes       no     yes  no         yes     yes    yes      no       4      3        2     2    3    3      10       7  8           10
                       GT3     T       4    2    health  services home   mother   1          3         0        no        yes    yes  yes        yes     yes    yes      yes      3      2        2     1    1    5      2        15 14          15

#Preparing X and Y variables

In [5]:
X = dataset.drop(columns=['G3'],axis=1)

KeyError: "['G3'] not found in axis"

In [6]:
X.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [8]:
print("Categories in 'school' variable:     ",end=" " )
print(dataset['school'].unique())

print("Categories in 'sex' variable:     ",end=" " )
print(dataset['sex'].unique())

print("Categories in 'age' variable:  ",end=" ")
print(dataset['age'].unique())



Categories in 'school' variable:      

KeyError: 'school'

In [9]:
y = dataset['G3']

KeyError: 'G3'

In [10]:
y

array(['G3', '6', '6', '10', '15', '10', '15', '11', '6', '19', '15', '9',
       '12', '14', '11', '16', '14', '14', '10', '5', '10', '15', '15',
       '16', '12', '8', '8', '11', '15', '11', '11', '12', '17', '16',
       '12', '15', '6', '18', '15', '11', '13', '11', '12', '18', '11',
       '9', '6', '11', '20', '14', '7', '13', '13', '10', '11', '13',
       '10', '15', '15', '9', '16', '11', '11', '9', '9', '10', '15',
       '12', '6', '8', '16', '15', '10', '5', '14', '11', '10', '10',
       '11', '10', '5', '12', '11', '6', '15', '10', '8', '6', '14', '10',
       '7', '8', '18', '6', '10', '14', '10', '15', '10', '14', '8', '5',
       '17', '14', '6', '18', '11', '8', '18', '13', '16', '19', '10',
       '13', '19', '9', '16', '14', '13', '8', '13', '15', '15', '13',
       '13', '8', '12', '11', '9', '0', '18', '0', '0', '12', '11', '0',
       '0', '0', '0', '12', '15', '0', '9', '11', '13', '0', '11', '0',
       '11', '0', '10', '0', '14', '10', '0', '12', '8', '13', '

In [11]:
#Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
X.shape

In [ ]:
#Splitting the dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

In [ ]:
#Create an Evaluate Function to give all metrics after model Training
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [ ]:
#Results
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

In [ ]:
#Linear Regression
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

In [ ]:
#Plot y_pred and y_test
plt.scatter(y_test,y_pred);
plt.xlabel('Actual');
plt.ylabel('Predicted');

In [ ]:
sns.regplot(x=y_test,y=y_pred,ci=None,color ='red');

In [ ]:
#Difference between Actual and Predicted Values
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df